In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer

In [2]:
# Load data from CSV files
calendar_df = pd.read_csv("C:\\Users\\Anup Jacob\\Downloads\\capstonegroup3\\calendar.csv")
listings_df = pd.read_csv("C:\\Users\\Anup Jacob\\Downloads\\capstonegroup3\\listings.csv") 
hosts_df = pd.read_csv("C:\\Users\\Anup Jacob\\Downloads\\capstonegroup3\\hosts.csv")


In [3]:
# Print column names of listings_df
print("Columns in listings_df:")
print(listings_df.columns)

Columns in listings_df:
Index(['listing_id', 'listing_url', 'name', 'description', 'latitude',
       'longitude', 'property_type', 'room_type', 'accomodates',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'host_id'],
      dtype='object')


In [4]:
# Data Understanding and Feature Creation

# Examine Calendar table
print(f"Calendar table shape: {calendar_df.shape}")
print(f"Unique listings in Calendar: {calendar_df['listing_id'].nunique()}")

# Clean price column and convert to float
calendar_df['price'] = calendar_df['price'].astype(str).str.replace('$', '').astype(float)

# Aggregate Calendar data by listing_id
calendar_agg_df = calendar_df.groupby('listing_id').agg(
    avg_price=('price', 'mean'),
    num_bookings=('available', lambda x: sum(x == 0)),
    avg_min_nights=('minimum_nights', 'mean'),
    avg_max_nights=('maximum_nights', 'mean')
).reset_index()

# Select necessary columns from Listings data
listings_cols = ['listing_id', 'bedrooms', 'beds', 'property_type', 'room_type']
listings_subset_df = listings_df[listings_cols]

# Join Listings data 
listings_join_df = pd.merge(listings_subset_df, calendar_agg_df, on='listing_id', how='left')

# Remove the merging step with hosts_df
df = listings_join_df

Calendar table shape: (319192, 8)
Unique listings in Calendar: 1749


In [5]:
# Data Quality Checks

# Check for missing values
print(f"Missing values per column:\n{df.isnull().sum()}")

# Descriptive statistics for numeric variables
print(f"Numeric variable statistics:\n{df.describe()}")

# Frequency counts for categorical variables
cat_vars = ['property_type', 'room_type']
for var in cat_vars:
    print(f"Frequency counts for {var}:\n{df[var].value_counts(normalize=True)}")
    
# Drop rows with missing target variable  
df = df.dropna(subset=['avg_price'])

Missing values per column:
listing_id          0
bedrooms          128
beds               41
property_type       0
room_type           0
avg_price           0
num_bookings        0
avg_min_nights      0
avg_max_nights      0
dtype: int64
Numeric variable statistics:
         listing_id     bedrooms         beds    avg_price  num_bookings  \
count  1.749000e+03  1621.000000  1708.000000  1749.000000   1749.000000   
mean   3.485506e+07     1.529303     2.220726   110.115098     84.827330   
std    1.523689e+07     1.091453     2.227509   184.006315     69.710269   
min    5.090400e+04     1.000000     1.000000    14.000000      0.000000   
25%    2.337286e+07     1.000000     1.000000    60.000000     12.000000   
50%    3.889829e+07     1.000000     2.000000    79.055000     87.000000   
75%    4.838954e+07     2.000000     3.000000   112.338710    153.000000   
max    5.398332e+07    20.000000    44.000000  5800.000000    205.000000   

       avg_min_nights  avg_max_nights  
count   